# Project 3. Finding names in a phone directory

## *Working with SQL in Python*

In the present example we have used a database containing all the clients of the only telecommunications provider in Cuba. The data was leaked in 2008 by an employee of the company. The leak was a big violation of customers' privacy, and it has been spread in different versions of mobile apps created by Cuban programmers. 

The journalism piece that could be created from this analysis would not identify personal data of the users, but the processing of the data below will show some of that information.

Our goal is to get a collection of Cuban first names, to determine which are more common. Government agencies and registries in Cuba don't publish this information periodically like other countries, so that is our motivation to use this data set in particular.

And since the resulting material wouldn't be making reference to any individuals in particular, just to groups of commonly used names, we don't think we would be contributing to further expose the customer's private information - if that is possible.

**Type of text analysed**: Customer database                 
**Text data description**: Relational database containing 4 tables and over 2 million clients              
**Type of project by data size**: Medium                      
**Data collection**: Database obtained from an app 

## SQL in Jupyter

We had many reasons to work with this database in IPython, and not in any other SQL client. For the courses we took online to learn SQL, we used SQLite3 in the command line, and Microsoft SQL Server Management Studio. 

One of the reasons to do this is that we also wanted to use the notebooks for this mini-project. Logically, the first thing we did was to do some research to find out if there was an SQL kernel that could be added to the Jupyter installation.

We didn't find one, but we found a very clever work around by [Catherine Devlin](https://github.com/catherinedevlin): she created an SQL client for Jupyter using `Magics`, the group of functions that allow to run code written in different languages within the same notebook.

![](https://s19.postimg.org/maznvmd0j/sql_client.jpg)

## SQL in Python

As you can see in the image above, once you install and load the extension, the use of SQL is very straight forward. But having already worked with SQL in the command line, we wanted to use this opportunity to learn the basics of the work with SQL in Python, with a view to being able to include SQL queries in our Python workflow in the future.

For this example, we are going to use the `sqlite3` module, a DB-API 2.0 interface for SQLite databases in Python.

### (1) Connecting to the database

Let's start by importing the module: (these initial steps follow the information in the [documentation for sqlite3](https://docs.python.org/3.5/library/sqlite3.html))

In [19]:
import sqlite3

Our database file is called `dirtelefonico.db3` and we have saved a copy in our working directory. To connect to the database, we create the variable `conn` and we assign to it the exectution of the `connect` method as follows:

In [20]:
conn = sqlite3.connect("dirtelefonico.db3")

Next, we apply the `cursor` method to that connection, and we store it in a variable, so that we can use that to run our different queries:

In [21]:
c = conn.cursor()

### (2) Exploring the data

Now that we have our cursor object (`c`), we can use that to run our queries. 

The first thing we need to do is to find the schema of the database, that is, learning how many tables it contains and what are their respective structures.

#### Tables comprising the database

We beging finding out the names of the tables:

In [22]:
c.execute("SELECT name FROM sqlite_master WHERE type='table'")
c.fetchall()

[('CENTER_DATOS_CLIENTE_CUBACEL',),
 ('CLASIFICACION POR SECTORES',),
 ('FIJOS NACIONALES',),
 ('Publicas',),
 ('name_dob2',),
 ('name_dob',)]

So the database includes 4 tables. Judging from the names of the tables, the last 2 could contain information about landlines and paid phones, respectively, but it's not really clear.

The best way to start exploring the tables is to get the names of the columns they contain.

#### Column names in each of the tables

In [23]:
# After trying the options we knew from SQL, we discovered that dot-commands don't work in this configuration, as explained
# in item 4 of the Command Line Shell For SQLite in this link https://www.sqlite.org/cli.html. Commands like 
# INFORMATION_SCHEMA.COLUMNS, or .schema, etc. wouldn't work

# after trying many options we found this method in Stack Overflow 
# http://stackoverflow.com/questions/11996394/is-there-a-way-to-get-a-schema-of-a-database-from-within-python

# Column names and type of content for table 1 (CENTER_DATOS_CLIENTE_CUBACEL) 
c.execute("PRAGMA table_info(CENTER_DATOS_CLIENTE_CUBACEL);")
c.fetchall()

[(0, 'NUMERO_TELEFONO', 'REAL', 0, None, 0),
 (1, 'IMSI', 'TEXT', 0, None, 0),
 (2, 'TIPO_NUMERO', 'TEXT', 0, None, 0),
 (3, 'USUARIO_TELEFONO', 'TEXT', 0, None, 0),
 (4, 'IDENTIFICACION', 'TEXT', 0, None, 0),
 (5, 'DIRECCION', 'TEXT', 0, None, 0),
 (6, 'NACIONALIDAD', 'TEXT', 0, None, 0),
 (7, 'PERSONA_AUTORIZADA', 'TEXT', 0, None, 0),
 (8, 'DIRECCION_PERSONA_AUTORIZADA', 'TEXT', 0, None, 0),
 (9, 'FECHA_ACTIVACION', 'DATE', 0, None, 0),
 (10, 'CLIENTE', 'REAL', 0, None, 0),
 (11, 'NOMBRE_CLIENTE', 'TEXT', 0, None, 0),
 (12, 'CLASIFICACION', 'REAL', 0, None, 0),
 (13, 'CLASIFICACION_CLIENTE', 'TEXT', 0, None, 0),
 (14, 'CONTRATO', 'REAL', 0, None, 0),
 (15, 'NOMBRE_CONTRATO', 'TEXT', 0, None, 0),
 (16, 'PROVINCIA', 'TEXT', 0, None, 0),
 (17, 'indice', 'INTEGER', 0, None, 0)]

In [24]:
# Column names and type of content for table 1 (CLASSIFICATION BY SECTOR) 
c.execute("PRAGMA table_info(`CLASIFICACION POR SECTORES`);")
c.fetchall()

[(0, 'SECTOR', 'TEXT', 0, None, 0), (1, 'CLASIFICACIÓN', 'TEXT', 0, None, 0)]

In [25]:
# Column names and type of content for table 1 (NATIONAL LANDLINES) 
c.execute("PRAGMA table_info(`FIJOS NACIONALES`);")
c.fetchall()

[(0, 'CENTROFACT', 'TEXT', 0, None, 0),
 (1, 'SECTOR', 'TEXT', 0, None, 0),
 (2, 'SERVICIOS', 'TEXT', 0, None, 0),
 (3, 'EXPR1', 'TEXT', 0, None, 0),
 (4, 'APELL1', 'TEXT', 0, None, 0),
 (5, 'APELL2', 'TEXT', 0, None, 0),
 (6, 'DIRECCION', 'TEXT', 0, None, 0),
 (7, 'INSC', 'TEXT', 0, None, 0),
 (8, 'INSN', 'TEXT', 0, None, 0),
 (9, 'INSK', 'TEXT', 0, None, 0),
 (10, 'ETC1', 'TEXT', 0, None, 0),
 (11, 'ETC2', 'TEXT', 0, None, 0),
 (12, 'RPTO', 'TEXT', 0, None, 0),
 (13, 'PUEBLO', 'TEXT', 0, None, 0),
 (14, 'TI', 'INT', 0, None, 0)]

In [26]:
# Column names and type of content for table 1 (Public) 
c.execute("PRAGMA table_info(Publicas);")
c.fetchall()

[(0, 'Número', 'TEXT', 0, None, 0),
 (1, 'Dirección', 'TEXT', 0, None, 0),
 (2, 'Municipio', 'TEXT', 0, None, 0),
 (3, 'Localidad', 'TEXT', 0, None, 0),
 (4, 'Reparto', 'TEXT', 0, None, 0),
 (5, 'Localización', 'TEXT', 0, None, 0),
 (6, 'Centro Atención', 'TEXT', 0, None, 0),
 (7, 'Ura', 'TEXT', 0, None, 0),
 (8, 'Zona', 'TEXT', 0, None, 0),
 (9, 'Cerradura', 'TEXT', 0, None, 0),
 (10, 'Inventario', 'TEXT', 0, None, 0),
 (11, 'CPopular', 'TEXT', 0, None, 0),
 (12, 'Organismo', 'TEXT', 0, None, 0),
 (13, 'Propietario', 'TEXT', 0, None, 0),
 (14, 'CI Prop', 'TEXT', 0, None, 0),
 (15, 'Circunscripción', 'TEXT', 0, None, 0),
 (16, 'Categ Clte', 'TEXT', 0, None, 0),
 (17, 'TipoServ', 'TEXT', 0, None, 0),
 (18, 'Sector', 'TEXT', 0, None, 0),
 (19, 'Cód Mercad', 'TEXT', 0, None, 0),
 (20, 'Categoría', 'TEXT', 0, None, 0),
 (21, 'Soporte', 'TEXT', 0, None, 0),
 (22, 'Entidad', 'TEXT', 0, None, 0),
 (23, 'Motivo Soc', 'TEXT', 0, None, 0)]

Most of the column names convey a clear idea of their content (it's easy to anticipate what the content of columns like *Number, Municipality, Owner, Last name* etc. is going to be; other names like *Lock, Inventory* and unknown accronyms like *Ura, IMSI* are obscure though).

Combining this with our knowledge of telephone services in Cuba, we can tell at this point that:
- table 1 contains information about mobile services (cell phones)
- no idea what table 2 is about (a junction table maybe?)
- table 3 contains information about landlines
- table 4 contains sinformation about paid phones

### Looking at samples of content

We can continue to explore the data by looking at a sample of the content, and the most logic way to do that is look at the first row of content of each table.

In [27]:
# Content of the first row in Table 1
c.execute("SELECT * FROM CENTER_DATOS_CLIENTE_CUBACEL")
c.fetchone()

(5352973132.0,
 '368010100352377',
 'Prepago',
 'ILEANA ANGELA PULDIN PLANCHART',
 '59100210372',
 'CALLE 281 NO18837 188 Y 184 BALUARTE ',
 'CUBANA',
 None,
 'CALLE 281 NO18837188 Y 184 BALUARTE',
 '2007-08-30',
 504369.0,
 'ILEANA ANGELA PULDIN PLANCHART',
 1.0,
 'A TITULO PERSONAL',
 703623.0,
 'ILEANA ANGELA PULDIN PLANCHART',
 'CIUDAD DE LA HABANA',
 None)

In [28]:
# Content of the first row in Table 2
c.execute("SELECT * FROM `CLASIFICACION POR SECTORES` limit 20")
c.fetchall()

[('AM', 'ALCANCIA MONEDERO'),
 ('PT', 'PARTICULAR'),
 ('SS', 'MININT'),
 ('MN', 'MININT'),
 ('MF', 'MINFAR'),
 ('ME', 'MINFAR'),
 ('LS', 'PARTICULAR'),
 ('LM', 'PARTICULAR')]

In [29]:
# Content of the first row in Table 3
c.execute("SELECT * FROM `FIJOS NACIONALES`")
c.fetchone()

('Pinar del Rio',
 'AF',
 '2197009',
 'Iglesia Pentecostal Luz del Mundo',
 None,
 None,
 '28 de Septiembre #53    P.del Rio',
 '28 de Septiembre',
 '53',
 None,
 None,
 None,
 None,
 'P.del Rio',
 0)

In [30]:
# Content of the first row in Table 4
c.execute("SELECT * FROM Publicas")
c.fetchone()

('7620017     ',
 'AVE 5TA ZONA 19                                                                                                                                                                                                                                                ',
 'Habana del Este',
 'ALAMAR                        ',
 'ZONA 19',
 'MERCADO (I)                                                                                                                                                                                                                                                    ',
 'ALAMAR        ',
 'Alamar',
 'Alamar2',
 '',
 '',
 'ALAMAR ESTE',
 '',
 '',
 '           ',
 '',
 'TELEF. PUB. MON. EXTERIOR',
 'AM',
 'AM',
 'TIENDAS DE VIVERES Y SUPERMERCADOS',
 '3',
 'PARED                    ',
 '',
 '')

This second step of the exploration confirms the ideas we proposed before about the content of the tables (and we still don't know what table 2 is about, we discarded the hypothesis of it being a junction or bridge table (used in many-to-many relations as a link between other tables). It's structue (only 2 columns) seemed to point to that possibility, but its non-unique content is not consistent with that idea.

### Ideas for stories

Although we started this search with names in mind, this preliminary exploration of the data points to other possible stories, base on calculations that can 


### (3) Finding the data for our story: extracting the names

If we take a look at the `USUARIO_TELEFONO` (CLIENT) list, we will see that it contains the following:

In [31]:
# Content of the first row in Table 1
c.execute("SELECT USUARIO_TELEFONO FROM CENTER_DATOS_CLIENTE_CUBACEL LIMIT 10")
c.fetchall()

[('ILEANA ANGELA PULDIN PLANCHART',),
 ('JORGE LUIS MARTINEZ HERNANDEZ',),
 ('IVAN MARIÑO PADRON',),
 ('JESSIE RODRIGUEZ LOPEZ',),
 ('HUMBERTO HERNANDEZ RAMOS',),
 ('GONZALO LAZARO COLINA DELGADO',),
 ('IRAIDA AGUSTINA BORREGO RODRIGUEZ',),
 ('ERNESTO RABEIRO MARIMON',),
 ('ANGEL GILBERTO GONZALEZ DIAZ',),
 ('MIGUEL ANGEL MIRANDA PEREZ',)]

As you may see, the column contains the name(s) and surnames of the clients. In Cuba, like in may other Spanish speaking countries, people are registered with two surnames: the first surname is the father's last name, and the second is the mother's last name. 

Single mothers may decide to give the child both their surnames, and there may be less common exceptions but the rule is that people have two.

That works really well for our example, becuase it gives us a relatively good way to extract just the names.

Let's explore the data:

In [32]:
# Counting the number of rows (clients)
c.execute("SELECT COUNT(USUARIO_TELEFONO) FROM CENTER_DATOS_CLIENTE_CUBACEL")
c.fetchall()

[(1018659,)]

In [33]:
# Counting the number of Cuban clients
c.execute("SELECT COUNT(USUARIO_TELEFONO) FROM CENTER_DATOS_CLIENTE_CUBACEL WHERE NACIONALIDAD= 'CUBANA';")
c.fetchall()

[(999683,)]

We are now going to create a separate table to make our transformations there, and preserve the original tables.

In [38]:
c.executescript("""CREATE TABLE name_dob AS
          SELECT USUARIO_TELEFONO, IDENTIFICACION 
          FROM CENTER_DATOS_CLIENTE_CUBACEL 
          WHERE NACIONALIDAD='CUBANA';
          """)
c.fetchall()

[]

In [ ]:
# Keep this lines around to drop and re-create name_dob when re-running the notebook
# c.execute("DROP TABLE name_dob")
# c.fetchall()

We do a quick check to make sure it contains what we expected:

In [39]:
c.execute("PRAGMA table_info(name_dob)")
c.fetchall()

[(0, 'USUARIO_TELEFONO', 'TEXT', 0, None, 0),
 (1, 'IDENTIFICACION', 'TEXT', 0, None, 0)]

In [40]:
c.execute("SELECT * FROM name_dob  LIMIT 10")
c.fetchall()

[('ILEANA ANGELA PULDIN PLANCHART', '59100210372'),
 ('JORGE LUIS MARTINEZ HERNANDEZ', '69100500084'),
 ('IVAN MARIÑO PADRON', '67021402488'),
 ('JESSIE RODRIGUEZ LOPEZ', '89101701859'),
 ('HUMBERTO HERNANDEZ RAMOS', '65101520104'),
 ('GONZALO LAZARO COLINA DELGADO', '64101802141'),
 ('IRAIDA AGUSTINA BORREGO RODRIGUEZ', '66050514514'),
 ('ERNESTO RABEIRO MARIMON', '73121900900'),
 ('ANGEL GILBERTO GONZALEZ DIAZ', '76011201449'),
 ('MIGUEL ANGEL MIRANDA PEREZ', '79072507504')]

We also want to make sure that the results for the foreigners were excluded, something we can do counting the content of the Clients column:

In [75]:
c.execute("SELECT COUNT(USUARIO_TELEFONO) FROM name_dob")
c.fetchall()

[(999683,)]

Now we can create a data frame to do our calculations:

In [43]:
from pandas import *

In [44]:
import pandas.io.sql as sql
import pandas as DataFrame
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html
table = pandas.read_sql("SELECT * FROM name_dob", conn)

In [79]:
table = table.rename(columns={'USUARIO_TELEFONO': 'Name', 'IDENTIFICACION': 'ID'})
table.head()

,Name,ID
0,ILEANA ANGELA PULDIN PLANCHART,59100210372
1,JORGE LUIS MARTINEZ HERNANDEZ,69100500084
2,IVAN MARIÑO PADRON,67021402488
3,JESSIE RODRIGUEZ LOPEZ,89101701859
4,HUMBERTO HERNANDEZ RAMOS,65101520104


In [80]:
import pandas as pd
table["Name"] = table["Name"].astype(str)
table.head()

,Name,ID
0,ILEANA ANGELA PULDIN PLANCHART,59100210372
1,JORGE LUIS MARTINEZ HERNANDEZ,69100500084
2,IVAN MARIÑO PADRON,67021402488
3,JESSIE RODRIGUEZ LOPEZ,89101701859
4,HUMBERTO HERNANDEZ RAMOS,65101520104


In [81]:
names_df = table

The following fuction will allow us to split the content of column `Name` into different columns, putting the surnames first:

In [48]:
name_splitting = lambda x: pd.Series([i for i in reversed(x.split(' '))])

Now we apply it to the column in question:

In [64]:
new_df = names_df['Name'].apply(name_splitting)

Let's take a look at the beginning and the end of our resulting data frame:

In [66]:
new_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,PLANCHART,PULDIN,ANGELA,ILEANA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HERNANDEZ,MARTINEZ,LUIS,JORGE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PADRON,MARIÑO,IVAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LOPEZ,RODRIGUEZ,JESSIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RAMOS,HERNANDEZ,HUMBERTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DELGADO,COLINA,LAZARO,GONZALO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RODRIGUEZ,BORREGO,AGUSTINA,IRAIDA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MARIMON,RABEIRO,ERNESTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DIAZ,GONZALEZ,GILBERTO,ANGEL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PEREZ,MIRANDA,ANGEL,MIGUEL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
new_df.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
1002783,GONZALEZ,ALVAREZ,RAMON,MARIO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002784,GARCIA,CRUZ,LESDIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002785,HURTADO,DIAZ,ANABEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002786,CEDEÑO,MESA,GRISEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002787,CASTILLO,HERNANDEZ,ESTANISLAO,OSVALDO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002788,GUERRA,VALLADARES,WILFREDO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002789,FIGUEREDO,CABRERA,NILDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002790,TELLERIA,GONZALEZ,MICHEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002791,PEREIRA,GAVEZ,DANEISY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002792,CARRASCO,DOMINGUEZ,YOSELIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we explained before, you may see that columns 0 and 1 contain the surnames, and the names are distributed as of column 2. We are only going to take care of columns 2 and 3 in this exercise. 

#### Column 2

In [87]:
list2 = new_df[2].tolist()
len(list2)

1002793

In [88]:
len(set(list2))

69313

#### Column 3

In [89]:
list3 = new_df[3].tolist()
len (list3)

1002793

In [90]:
len(set(list3))

47709

#### Merging both columns

In [92]:
all_the_names = list2 + list3
len(all_the_names)

2005586

In [93]:
len(set(all_the_names))

95767

In [99]:
from collections import Counter
Counter(all_the_names).most_common(20)

[(nan, 604660),
 ('LUIS', 21058),
 ('MARIA', 19573),
 ('CARIDAD', 16541),
 ('DE', 15582),
 ('JOSE', 13067),
 ('LA', 12935),
 ('CARLOS', 12862),
 ('RODRIGUEZ', 12289),
 ('', 11996),
 ('ALBERTO', 11496),
 ('MANUEL', 11256),
 ('JORGE', 11128),
 ('ANTONIO', 10883),
 ('GONZALEZ', 9959),
 ('PEREZ', 9630),
 ('JUAN', 8524),
 ('JESUS', 7841),
 ('HERNANDEZ', 7822),
 ('MIGUEL', 7514)]

Although the results show that some surnames got into our list, we have a preliminary (and ultimately publishable) answer to our question: LUIS and MARIA are the most common names in the list of clients of ETECSA.

## Conclusions

This is only a first approach to this database, but we know we still need to refine our calculations. This may not be the most common type of document we get in newsrooms, but the experience with this one may be a good reference to work with a database including less private information in the future.

## References

- Python 3.5.2 Documentation (2015), DB-API 2.0 interface for SQLite databases. Available at: https://docs.python.org/3.5/library/sqlite3.html Accessed on 20 Aug. 2016

- Stack Overflow (2012),*Is there a way to get a schema of a database from within python?*, available at
http://stackoverflow.com/questions/11996394/is-there-a-way-to-get-a-schema-of-a-database-from-within-python

- Devlin, Catherin (2016), *ipython-sql*, available at: https://github.com/catherinedevlin/ipython-sql